In [1]:
#import pyfair
from pyfair.model.model import FairModel
from pyfair.model.meta_model import FairMetaModel

import matplotlib
%matplotlib inline

import numpy as np
import pandas as pd

In [2]:
import json

with open('./serialized_model.json', 'r') as f:
    j = json.loads(f.read())

FairModel.read_json('./serialized_model.json')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
model1 = pyfair.FairModel(name="HA1", n_simulations=10_000)
model1.input_data('Loss Event Frequency', low=20, mode=100, high=900)
model1.input_data('Primary Loss', low=3_000_000, mode=3_500_000, high=5_000_000)
model1.input_multi_data('Secondary Loss', {
    'Secondary Loss Event Frequency': {
        'Reputational': {
            'low': .01,
            'mode': .80,
            'high': .99,
        }
    },
    'Secondary Loss Event Magnitude' : {
        'Reputational': {
            'low': 10_000,
            'mode': 20_000,
            'high': 100_000,
        }
    }
})

model1.calculate_all()

model2 = pyfair.FairModel(name="HA2", n_simulations=10_000)
model2.bulk_import_data({
    'Loss Event Frequency':    {'mean':.5, 'stdev':.1},
    'Loss Magnitude': {'low': 2_000_000_000, 'mode': 3_000_000_000, 'high': 5_000_000_000}
})
model2.calculate_all()

model3 = pyfair.FairModel(name="HA3", n_simulations=10_000)
model3.input_data('Loss Event Frequency', low=10, mode=15, high=16)
model3.input_data('Loss Magnitude', low=500_000_000, mode=500_900_000, high=600_000_000)
model3.calculate_all()

fsr = pyfair.FairSimpleReport([model1])
fsr.to_html('output.html')

In [ ]:
mm = pyfair.FairMetaModel(name='My Meta Model!', models=[model1, model2])
mm.calculate_all()



#fsr = pyfair.FairSimpleReport([model2, mm])
#fsr.to_html('output.html')

print(mm.to_json())

with open('./serialized_model.json', 'w') as f:
    f.write(mm.to_json())
    
with open('./serialized_model.json', 'r') as f:
    model = FairMetaModel.read_json(f.read())
    
model

# Components

<section header and metadata>

<section overview>

* Summary General
    * Summary Table (mean, stdev, low, high, n)
    * hist
    * Exceedence curves
* Summary Comparison
    * Summary Table (mean, stdev, low, high, n)
    * Summary Table 2
    * Combined Hist
    * Exceedence curves
    
<section parameters>

* Model Detail
    * Parameter table with distributions
* Metamodel Detail
    * Risk table with stacked area
    
<section> and source code
* JSON

# Reports
* Compare
* Single
* Metamodel Drilldown

In [ ]:
from pyfair.utility.factory import FairModelFactory

static = {
    "Loss Event Frequency": {
        "low": 20,
        "mode": 100,
        "high": 900
    },
}

variable = {
    'Model 1': {
        "Loss Magnitude": {
            "low": 3000000,
            "mode": 3500000,
            "high": 5000000
        }
    },
    'Model 2': {
        "Loss Magnitude": {
            "low": 300000,
            "mode": 350000,
            "high": 500000
        },
    }
}
    
factory = FairModelFactory(static)

factory.generate_from_partials(variable)

In [ ]:
from pyfair.report.distribution import FairDistributionCurve

fdc = FairDistributionCurve([model1, model2, model3])
#fig, ax = fdc.generate_image()

fig, ax = fdc.generate_image()
fdc.generate_icon('HA1', 'Risk')

In [ ]:
from pyfair.report.exceedence import FairExceedenceCurves

fpc = FairExceedenceCurves([model1, model2, model3])
fig1, ax1 = fpc.generate_image()

In [ ]:
import numpy as np